In [30]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd

In [399]:
n = 6 * 12
x = np.arange(n)
y = np.abs(np.random.randn(n))
y_cs = np.cumsum(y) * 1000

data = pd.DataFrame({"x": x, "y": y, "A": y_cs, "B": 2 * y_cs, "C": 3 * y_cs})

In [400]:
def sum_lines_traces(data, ycols, ycol_highlight, left, right):
    hover_template = "<b>Cumulative Cost</b>: $%{y:,.0f}"

    traces = []
    for column in ycols:
        if column == ycol_highlight:
            # plot from 0 to shade_l
            data_left = data.loc[data.index < left]
            data_right = data.loc[data.index > right]
            data_middle = data.loc[(data.index >= left) & (data.index <= right)]
            for d in [data_left, data_right, data_middle]:
                opacity = 1 if d is data_middle else 0.2
                traces.append(
                    go.Scatter(
                        x=d.index,
                        y=d[column],
                        mode="lines",
                        line=dict(color="rgba(65,40,200,1)"),
                        showlegend=False,
                        opacity=opacity,
                        hovertemplate=hover_template,
                        name=column,
                    )
                )

        else:
            traces.append(
                go.Scatter(
                    x=data.index,
                    y=data[column],
                    mode="lines",
                    line=dict(color="rgba(60,60,60,0.25)"),
                    showlegend=False,
                    hovertemplate=hover_template,
                    name=column,
                )
            )

    # add maker at left, right
    y_l = data.loc[data.index == left, ycol_highlight].values[0]
    y_r = data.loc[data.index == right, ycol_highlight].values[0]

    traces.append(
        go.Scatter(
            x=[left, right],
            y=[y_l, y_r],
            mode="markers",
            marker=dict(color="rgba(95,40,200,1)", size=6),
            showlegend=False,
            hoverinfo="skip",
        )
    )

    return traces

In [401]:
def vertical_traces(data, ycol_highlight, left, right):

    traces = []
    # plot from 0 to shade_l
    data_left = data.loc[data.index < left, [ycol_highlight]]
    data_right = data.loc[data.index > right, [ycol_highlight]]
    data_middle = data.loc[
        (data.index >= left) & (data.index <= right), [ycol_highlight]
    ]
    for d in [data_left, data_right, data_middle]:
        opacity = 1 if d is data_middle else 0.2
        traces.append(
            go.Scatter(
                x=[0] * len(d),
                y=d[ycol_highlight],
                mode="lines",
                line=dict(color="rgba(65,40,200,1)"),
                showlegend=False,
                opacity=opacity,
                hoverinfo="skip",
            )
        )

    # add marker at left, right
    y_l = data_middle[ycol_highlight].min()
    y_r = data_middle[ycol_highlight].max()

    traces.append(
        go.Scatter(
            x=[0, 0],
            y=[y_l, y_r],
            mode="markers",
            marker=dict(color="rgba(95,40,200,1)", size=6),
            showlegend=False,
            hovertemplate="<br><b>Cumulative Cost</b>: $%{y:,.0f}",
            name=ycol_highlight,
        )
    )

    traces.append(
        go.Scatter(
            x=[0],
            y=[(y_r + y_l) / 2],
            # text=[f"${y_r - y_l:,.0f}"],
            # textfont=dict(size=14, color="rgba(95,40,200,1)", weight="bold"),
            # textposition="middle center",
            hoverinfo="skip",
            mode="text",
            showlegend=False,
        )
    )

    return traces

In [402]:
def plot_trend(data, xcol, columns_included, column_highlight, left, right):
    # create a copy of data to avoid modifying the original
    data = data.copy()
    # make xcol the index
    data.set_index(xcol, inplace=True)

    # offset data to start at left
    data[columns_included] = (
        data[columns_included] - data.loc[left, columns_included]
    )
    # subplots
    fig = make_subplots(
        rows=1,
        cols=2,
        column_widths=[8, 1],
        row_heights=[8],
        shared_yaxes=True,
        horizontal_spacing=0.02,
    )

    # plot lines in the main plot

    y_l = data.loc[left, column_highlight]
    y_r = data.loc[right, column_highlight]

    duration_sum = y_r - y_l

    main_traces = sum_lines_traces(
        data, columns_included, column_highlight, left, right
    )

    vert_traces = vertical_traces(data, column_highlight, left, right)

    for trace in main_traces:
        fig.add_trace(trace, row=1, col=1)

    for trace in vert_traces:
        fig.add_trace(trace, row=1, col=2)

    for y in [y_l, y_r]:
        fig.add_hline(y=y, line_dash="dot", row=1, col="all", opacity=0.2)

    # add anotation to the right plot
    fig.add_annotation(
        x=0,
        y=(y_r + y_l) / 2,
        text=f"${duration_sum:,.0f}",
        showarrow=False,
        font=dict(size=20, color="rgba(95,40,200,1)", weight="bold"),
        xshift=15,
        yshift=0,
        row=1,
        col=2,
        textangle=90,
    )

    # add title as annotation to the left plot
    fig.add_annotation(
        text=f"Cumulative Cost of Childcare<br>${duration_sum:,.0f}",
        font=dict(
            size=20,
            color="rgba(95,40,200,1)",
            weight="bold",
            family="Helvetica",
        ),
        xref="x domain",
        yref="y domain",
        x=0,  # Left edge of the subplot
        y=1,  # Top edge of the subplot
        xshift=20,
        yshift=-15,
        xanchor="left",
        yanchor="top",
        showarrow=False,
        align="left",
    )

    # hide tick labels from the right plot
    fig.update_xaxes(
        range=[-0.1, 0.1],
        row=1,
        col=2,
        showgrid=False,  # This hides the vertical gridlines
        tickformat="%d-%b",  # This formats the tick labels (e.g., "01-Jan")
    )

    # update layout

    # update layout
    fig.update_layout(
        font=dict(size=14),
        width=800,
        height=600,
        # ymin
        yaxis=dict(
            range=[
                data[columns_included].min().min(),
                data[columns_included].max().max(),
            ],
            showgrid=False,
            tickformat="$,.0f",  # format yaxis labels to show dollar sign
        ),
        xaxis=dict(
            showgrid=False,
        ),
        xaxis_title="Age (Month)",
        hovermode="x",
    )

    # turn off grid
    for i in range(1, 3):
        fig.update_yaxes(showgrid=False, row=1, col=i)
        fig.update_xaxes(showgrid=False, row=1, col=i)

    fig.update_xaxes(showticklabels=False, row=1, col=2)

    fig.show()

In [403]:
plot_trend(data, "x", ["A", "B", "C"], "B", 6, 4 * 12 + 6)

## One subplot


In [404]:
def vertical_traces(data, ycol_highlight, left, right):

    traces = []
    # plot from 0 to shade_l
    data_left = data.loc[data.index < left, [ycol_highlight]]
    data_right = data.loc[data.index > right, [ycol_highlight]]
    data_middle = data.loc[
        (data.index >= left) & (data.index <= right), [ycol_highlight]
    ]
    for d in [data_left, data_right, data_middle]:
        opacity = 1 if d is data_middle else 0.2
        traces.append(
            go.Scatter(
                x=[0] * len(d),
                y=d[ycol_highlight],
                mode="lines",
                line=dict(color="rgba(65,40,200,1)"),
                showlegend=False,
                opacity=opacity,
                hoverinfo="skip",
            )
        )

    # add marker at left, right
    y_l = data_middle[ycol_highlight].min()
    y_r = data_middle[ycol_highlight].max()

    traces.append(
        go.Scatter(
            x=[0, 0],
            y=[y_l, y_r],
            mode="markers",
            marker=dict(color="rgba(95,40,200,1)", size=6),
            showlegend=False,
            hovertemplate="<br><b>Cumulative Cost</b>: $%{y:,.0f}",
            name=ycol_highlight,
        )
    )

    traces.append(
        go.Scatter(
            x=[1],
            y=[y_r],
            text=[f"${y_r - y_l:,.0f}"],
            textfont=dict(size=16, color="rgba(95,40,200,1)", weight="bold"),
            textposition="middle right",
            hoverinfo="skip",
            mode="text",
            showlegend=False,
        )
    )

    return traces

In [468]:
def plot_trend(data, xcol, columns_included, column_highlight, left, right):
    # create a copy of data to avoid modifying the original
    data = data.copy()
    # make xcol the index
    data.set_index(xcol, inplace=True)

    # offset data to start at left
    data[columns_included] = (
        data[columns_included] - data.loc[left, columns_included]
    )
    # subplots
    fig = make_subplots(
        rows=1,
        cols=1,
        column_widths=[8],
        row_heights=[8],
        shared_yaxes=True,
        horizontal_spacing=0.02,
    )

    # plot lines in the main plot

    y_l = data.loc[left, column_highlight]
    y_r = data.loc[right, column_highlight]

    duration_sum = y_r - y_l

    main_traces = sum_lines_traces(
        data, columns_included, column_highlight, left, right
    )

    vert_traces = vertical_traces(data, column_highlight, left, right)

    for trace in main_traces:
        fig.add_trace(trace, row=1, col=1)

    for trace in vert_traces:
        fig.add_trace(trace, row=1, col=1)

    # for y in [y_l, y_r]:
    #     fig.add_hline(y=y, line_dash="dot", row=1, col="all", opacity=0.2)

    # # add title as annotation to the left plot
    # fig.add_annotation(
    #     text=f"Cumulative Cost of Childcare<br>${duration_sum:,.0f}",
    #     font=dict(
    #         size=20,
    #         color="rgba(95,40,200,1)",
    #         weight="bold",
    #         family="Helvetica",
    #     ),
    #     xref="x domain",
    #     yref="y domain",
    #     x=0,  # Left edge of the subplot
    #     y=1,  # Top edge of the subplot
    #     xshift=20,
    #     yshift=-15,
    #     xanchor="left",
    #     yanchor="top",
    #     showarrow=False,
    #     align="left",
    # )

    # update layout
    fig.update_layout(
        font=dict(size=14),
        width=800,
        height=600,
        # ymin
        yaxis=dict(
            range=[
                data[columns_included].min().min(),
                data[columns_included].max().max(),
            ],
            showgrid=True,
            tickformat="$~s",
            # ticklabelposition="inside",
            gridcolor="lightgray",
            gridwidth=1,
            griddash="dash",
            zeroline=False,
            showline=False,
            showticklabels=True,
            tickmode="array",
            ticklabelposition="inside top",
            tickfont=dict(color="gray", size=15),
        ),
        xaxis=dict(
            showgrid=False,
            ticks="inside",
            range=[-10, 72],
            tickvals=[0, 10, 20, 30, 40, 50, 60, 70],
            ticktext=[
                "0 <i>months</i>",
                "10",
                "20",
                "30",
                "40",
                "50",
                "60",
                "70",
            ],
            tickangle=0,
            tickfont=dict(size=14),
            showline=True,
            linewidth=1,
            linecolor="black",
        ),
        # xaxis_title="Age (Month)",
        hovermode="x",
        plot_bgcolor="white",
        margin=dict(l=0, r=20, t=30, b=60),
    )

    fig.show()

In [469]:
# move labels inside and make them 20k etc, add dashed horizontal lines for tickets
# annotate the marker for the total
# and notches

In [470]:
plot_trend(data, "x", ["A", "B", "C"], "B", 6, 4 * 12 + 6)

# Reference


In [415]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Sample data (you'd replace this with your actual data)
years = list(range(1920, 2021, 20))
chicken = [15, 15, 25, 40, 60, 90]
beef = [50, 60, 70, 75, 65, 55]
pork = [55, 50, 60, 50, 50, 45]
cheese = [4, 5, 8, 15, 30, 38]
yogurt = [0, 0, 2, 5, 10, 14]

# Create the figure
fig = make_subplots(rows=1, cols=1)

# Add traces for each food type
fig.add_trace(
    go.Scatter(x=years, y=chicken, name="Chicken", line=dict(color="#FF7F0E"))
)
fig.add_trace(
    go.Scatter(x=years, y=beef, name="Beef", line=dict(color="#BCBCBC"))
)
fig.add_trace(
    go.Scatter(x=years, y=pork, name="Pork", line=dict(color="#DBDBDB"))
)
fig.add_trace(
    go.Scatter(x=years, y=cheese, name="Cheese", line=dict(color="#FFC000"))
)
fig.add_trace(
    go.Scatter(x=years, y=yogurt, name="Yogurt", line=dict(color="#E5E5E5"))
)

# Update layout
fig.update_layout(
    title={
        "text": "Americans' Changing Diet<br><span style='font-size:14px;color:gray'>80 pounds per person</span>",
        "y": 0.95,
        "x": 0.05,
        "xanchor": "left",
        "yanchor": "top",
    },
    xaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=True,
        showticklabels=True,
        range=[1920, 2020],
        dtick=20,
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=1,
        griddash="dash",
        zeroline=False,
        showline=False,
        showticklabels=True,
        tickmode="array",
        tickvals=[20, 40, 60, 80],
        ticktext=["20", "40", "60", "80"],
        ticklabelposition="inside top",
        range=[0, 90],
        tickfont=dict(color="gray"),
    ),
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=1.02),
    margin=dict(r=100),  # Add right margin for legend
    plot_bgcolor="white",
)

fig.show()

## Old Code


In [ ]:
def add_vrect(fig, l, r, x_min, x_max):
    fig.add_vrect(
        x0=x_min,
        x1=l,
        fillcolor="black",
        opacity=0.2,
        line_width=0,
        row=1,
        col=1,
    )
    fig.add_vrect(
        x0=r,
        x1=x_max,
        fillcolor="black",
        opacity=0.2,
        line_width=0,
        row=1,
        col=1,
    )


def add_hrect(fig, b, t, y_min, y_max):
    fig.add_hrect(
        y0=y_min,
        y1=b,
        fillcolor="black",
        opacity=0.2,
        line_width=0,
        row=1,
        col=1,
    )
    fig.add_hrect(
        y0=t,
        y1=y_max,
        fillcolor="black",
        opacity=0.2,
        line_width=0,
        row=1,
        col=1,
    )


def add_lines(fig, l, r, b, t):
    fig.add_vline(
        x=l, line_width=1, line_dash="dot", line_color="black", opacity=0.5
    )

    fig.add_vline(
        x=r, line_width=1, line_dash="dot", line_color="black", opacity=0.5
    )

    fig.add_hline(
        y=b, line_width=1, line_dash="dot", line_color="black", opacity=0.5
    )

    fig.add_hline(
        y=t, line_width=1, line_dash="dot", line_color="black", opacity=0.5
    )